# Analyse des interventions non facturées

Ce notebook permet d'analyser les interventions non facturées dans Notion.

In [1]:
import pandas as pd
import datetime

from src.notion_module.analyse import from_interventions_to_datas
from src.notion_module.core import query_unbilled_entries
from src.notion_module.core import create_invoice_page

In [2]:
def etape_2 ():
    """
        Cette fonction permet de récupérer les interventions non facturées
        dans Notion et de les transformer en DataFrame pour analyse.
        Pour ignorer le statut de facturation, on peut passer le dernier paramètre à None
    """
    interventions = query_unbilled_entries("2025-01-01", "2025-01-30", None)

    datas = from_interventions_to_datas(interventions)

    df = pd.DataFrame(datas)

    # Conversion de la date en datetime
    df['Date de début'] = pd.to_datetime(df['Date de début'])

    # Ajout colonne mois
    df['Mois'] = df['Date de début'].dt.to_period('M')

    # Calcul somme à facturer
    df['Somme à facturer'] = df['Nombre heures'] * df['Tarif horaire']

    print(df.head())

    return df

dataframes = etape_2()

                Ville            Ecole Classe  Nombre heures  Tarif horaire  \
0  Le Kremlin-Bicêtre  EPITECH Digital     M1            7.0             10   
1  Le Kremlin-Bicêtre  EPITECH Digital     M1            7.0             10   
2               Paris              ECE     M1            7.0             10   
3               Paris              ECE     M1            7.0             10   
4  Le Kremlin-Bicêtre  EPITECH Digital     M1            7.0             10   

   Facturé Date de début     Mois  Somme à facturer  
0    False    2025-03-05  2025-03              70.0  
1    False    2025-03-05  2025-03              70.0  
2     True    2025-03-05  2025-03              70.0  
3     True    2025-03-19  2025-03              70.0  
4    False    2025-03-19  2025-03              70.0  


In [3]:
def by_city(df):
    df_ville = df.groupby('Ville').agg(
        nb_ecoles=('Ecole', 'nunique'),
        nb_heures=('Nombre heures', 'sum'),
        somme_a_facturer=('Somme à facturer', 'sum')
    ).reset_index()

    print(df_ville)

by_city(dataframes)

                Ville  nb_ecoles  nb_heures  somme_a_facturer
0                Evry          1       42.5             327.5
1  Le Kremlin-Bicêtre          1       21.0             210.0
2               Paris          2       53.0             725.0


In [4]:
def by_school_and_class(df):
    df_ecole_classe = df.groupby(['Ecole', 'Classe']).agg(
        nb_heures=('Nombre heures', 'sum'),
        somme_a_facturer=('Somme à facturer', 'sum')
    ).reset_index()

    print(df_ecole_classe)

by_school_and_class(dataframes)

                 Ecole    Classe  nb_heures  somme_a_facturer
0                  ECE   BTS SIO        3.5              52.5
1                  ECE        L3       16.0             240.0
2                  ECE        M1       14.0             140.0
3               ENSIIE        L3       23.0             167.5
4               ENSIIE  M1 Maths       19.5             160.0
5      EPITECH Digital        M1       21.0             210.0
6  NEXA Digital School   BTS SIO       19.5             292.5


In [5]:
def by_month(df):
    # On distingue passé vs futur (par rapport à aujourd'hui)
    aujourdhui = pd.to_datetime(datetime.datetime.today().date())

    df['periode'] = df['Date de début'].apply(lambda d: 'Passé' if d < aujourdhui else 'Futur')

    df_mois_periode = df.groupby(['Mois', 'periode']).agg(
        nb_heures=('Nombre heures', 'sum')
    ).reset_index()

    print(df_mois_periode)

by_month(dataframes)

      Mois periode  nb_heures
0  2025-03   Passé       35.0
1  2025-04   Passé       14.0
2  2025-05   Passé       14.0
3  2025-07   Futur       22.0
4  2025-08   Futur       31.5


In [6]:
def etape_3():
    """
        Cette fonction permet de créer une page de facture dans Notion
        à partir des données d'interventions non facturées.
    """
    interventions = query_unbilled_entries("2025-01-01", "2025-01-30", None)

    # On crée la page de facture
    print(create_invoice_page(
        "Test",
        interventions,
        "2025-01",
        "Alexandre2",
    ))

etape_3()

{'object': 'page', 'id': '2112cdbb-4757-81f4-a3dc-e994314e0411', 'created_time': '2025-06-13T12:06:00.000Z', 'last_edited_time': '2025-06-13T12:06:00.000Z', 'created_by': {'object': 'user', 'id': '3ad9a6c5-500e-4a6b-addf-a7e3896a77d6'}, 'last_edited_by': {'object': 'user', 'id': '3ad9a6c5-500e-4a6b-addf-a7e3896a77d6'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '2102cdbb-4757-8110-a76f-cb2284e29df3'}, 'archived': False, 'in_trash': False, 'properties': {'Mois': {'id': 'Jh%5EE', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': 'June 2025', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'June 2025', 'href': None}]}, 'Invoice Number': {'id': 'NoqM', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': 'Alexandre2', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'und